In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

/home/ozkan/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv('mesrur2.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144788 entries, 0 to 144787
Columns: 139 entries, SellerCode to Payment1st3Months_F
dtypes: float64(36), int64(103)
memory usage: 153.5 MB


In [4]:
data.head()

,SellerCode,FollowingFlag,citycode,GenderCode,CustomerAge,CustomerAgeGroup,LoanCountasGuarantor,LoanCountasGuarantorGroup,DistinctGuarantorCount,ProtocolCount,...,UnresolvedCheques_AcqUPB,UnresolvedCGoodsFinance_AcqUPB,UnresolvedCreditCard_CurLoanTRY3,UnresolvedHousingCredit_CurLoanTRY3,UnresolvedConsumerCredit_CurLoanTRY3,UnresolvedVehicleCredit_CurLoanTRY3,UnresolvedKMH_CurLoanTRY3,UnresolvedCheques_CurLoanTRY3,UnresolvedCGoodsFinance_CurLoanTRY3,Payment1st3Months_F
0,44,0,41.0,1.0,44.0,45,0,0,0,0,...,0.0,0.0,0.00,0.0,86854.60,0.0,0.00,0.0,0.0,0
1,43,1,38.0,1.0,29.0,30,0,0,0,0,...,0.0,0.0,0.00,0.0,8944.04,0.0,200.00,0.0,0.0,0
2,12,0,1.0,1.0,36.0,40,0,0,0,0,...,0.0,0.0,3469.47,0.0,0.00,0.0,0.00,0.0,0.0,0
3,15,0,NaN,1.0,29.0,30,0,0,0,0,...,0.0,0.0,1869.24,0.0,0.00,0.0,829.07,0.0,0.0,0
4,43,0,6.0,1.0,27.0,30,0,0,0,3,...,0.0,0.0,1217.40,0.0,0.00,0.0,0.00,0.0,0.0,0


In [5]:
y = data.iloc[:,-1]
del data['Payment1st3Months_F']

In [6]:
x_train1, x_test, y_train1, y_test = train_test_split(data, y, test_size=0.33, random_state=42, stratify=y)

In [10]:
split = 70000
x_train, y_train, x_valid, y_valid = x_train1[:split], y_train1[:split], x_train1[split:], y_train1[split:]

In [11]:
d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

In [ ]:
params = {}
params['eta'] = 0.02
params['objective'] = 'reg:linear'
params['eval_metric'] = 'mae'
params['max_depth'] = 4
params['silent'] = 1

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=100, verbose_eval=10)


[0]	train-mae:0.491706	valid-mae:0.491751
Multiple eval metrics have been passed: 'valid-mae' will be used for early stopping.

Will train until valid-mae hasn't improved in 100 rounds.
[10]	train-mae:0.417358	valid-mae:0.417796
[20]	train-mae:0.356558	valid-mae:0.357324
[30]	train-mae:0.306844	valid-mae:0.307917
[40]	train-mae:0.266186	valid-mae:0.267518
[50]	train-mae:0.232927	valid-mae:0.234508
[60]	train-mae:0.205707	valid-mae:0.2075
[70]	train-mae:0.18343	valid-mae:0.185428
[80]	train-mae:0.165193	valid-mae:0.16737
[90]	train-mae:0.150244	valid-mae:0.152584
[100]	train-mae:0.138005	valid-mae:0.140498
[110]	train-mae:0.127977	valid-mae:0.130616
[120]	train-mae:0.119759	valid-mae:0.122538
[130]	train-mae:0.113019	valid-mae:0.115922
[140]	train-mae:0.107486	valid-mae:0.110504
[150]	train-mae:0.102938	valid-mae:0.10606
[160]	train-mae:0.099198	valid-mae:0.102421
[170]	train-mae:0.096127	valid-mae:0.099449
[180]	train-mae:0.0936	valid-mae:0.097015
[190]	train-mae:0.09153	valid-mae:0.09

In [24]:
d_test = xgb.DMatrix(x_test)
p_test = clf.predict(d_test)

In [21]:
accuracy = accuracy_score(y_test, p_test.round())
matrix = confusion_matrix(y_test, p_test.round())nked

In [20]:
accuracy

0.9523032167597999

In [22]:
matrix

array([[45486,    21],
       [ 2258,    16]])